In [21]:
### Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf 
import talib


In [22]:
### Load in Stock Data
tickers = 'GOOG'
start_date = '2024-01-01'

def import_stock_data(tickers, start_date):
    # Check if tickers is a list or a single ticker string
    if isinstance(tickers, list):
        data = yf.download(tickers, start=start_date)[['Close', 'High', 'Low']]
    else:
        data = yf.download(tickers, start=start_date)[['Close', 'High', 'Low']]
        data = pd.DataFrame(data)

    # Reset index to make 'Date' a column instead of index
    data.reset_index(inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])

    return data

stock_data = import_stock_data(tickers, start_date)
stock_data

[*********************100%%**********************]  1 of 1 completed


,Date,Close,High,Low
0,2024-01-02,139.559998,140.615005,137.740005
1,2024-01-03,140.360001,141.089996,138.429993
2,2024-01-04,138.039993,140.634995,138.009995
3,2024-01-05,137.389999,138.809998,136.850006
4,2024-01-08,140.529999,140.639999,137.880005
...,...,...,...,...
71,2024-04-15,156.330002,160.830002,156.149994
72,2024-04-16,156.000000,157.229996,155.050003
73,2024-04-17,156.880005,158.681000,156.134995
74,2024-04-18,157.460007,158.485001,156.210007


In [23]:
### Calculate ADX
def calculate_ADX(data, window):
    # Calculate True Range (TR)
    data['TR'] = pd.concat([data['High'], data['Close'].shift()], axis=1).max(axis=1) - pd.concat([data['Low'], data['Close'].shift()], axis=1).min(axis=1)

    # Calculate the Directional Movement (DM +/-) = Current High/Low - Previous High/Low (if both are positive, otherwise zero)
    for i in range(1, len(data)):
        # Build positive and negative DM columns
        pdm = data.iloc[i]['High'] - data.iloc[i - 1]['High']
        ndm = data.iloc[i - 1]['Low'] - data.iloc[i]['Low']
    
        data.at[i, 'PDM'] = pdm if pdm > 0 else 0
        data.at[i, 'NDM'] = abs(ndm) if ndm > 0 else 0

    # Smooth TR, PDM, and NDM using 14-period EMA
    data['TR_Smooth'] = data['TR'].rolling(window = window).mean()
    data['PDM_Smooth'] = data['PDM'].rolling(window = window).mean()
    data['NDM_Smooth'] = data['NDM'].rolling(window = window).mean()

    # Calculate the Directional Indicators (DI+ and DI-) = (PDM / Smoothed TR) * 100 and = (NDM / Smoothed TR) * 100
    data['DI+'] = (data['PDM'] / data['TR_Smooth']) * 100
    data['DI-'] = (data['NDM'] / data['TR_Smooth']) * 100

    # Calculate the Directional Index (DX) = (abs(DI+ - DI-) / (DI+ + DI-)) * 100
    data['DX'] = data['DX'] = (abs(data['DI+'] - data['DI-']) / (data['DI+'] + data['DI-'])) * 100

    # Calculate Average Directional Index (ADX) = 14-period EMA of DX
    data['ADX'] = data['DX'].rolling(window = window).mean()

    return data

# Function Call
single_ticker = calculate_ADX(stock_data, window = 14)
single_ticker

,Date,Close,High,Low,TR,PDM,NDM,TR_Smooth,PDM_Smooth,NDM_Smooth,DI+,DI-,DX,ADX
0,2024-01-02,139.559998,140.615005,137.740005,2.875000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-03,140.360001,141.089996,138.429993,2.660004,0.474991,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-04,138.039993,140.634995,138.009995,2.625000,0.000000,0.419998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-05,137.389999,138.809998,136.850006,1.959991,0.000000,1.159988,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-08,140.529999,140.639999,137.880005,3.250000,1.830002,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2024-04-15,156.330002,160.830002,156.149994,4.680008,0.000000,2.450012,3.199427,1.087429,0.488644,0.00000,76.576595,100.0,NaN
72,2024-04-16,156.000000,157.229996,155.050003,2.179993,0.000000,1.099991,3.200141,0.962857,0.567215,0.00000,34.373201,100.0,NaN
73,2024-04-17,156.880005,158.681000,156.134995,2.681000,1.451004,0.000000,3.208784,1.066500,0.502930,45.21975,0.000000,100.0,NaN
74,2024-04-18,157.460007,158.485001,156.210007,2.274994,0.000000,0.000000,3.275569,1.066500,0.502930,0.00000,0.000000,NaN,NaN
